In [ ]:
# Inicialize o Otter
import otter
grader = otter.Notebook("project3.ipynb")

# Projeto 3: Classificação de Filmes

Bem-vindo ao terceiro projeto do Data 8! Você construirá um modelo de classificação que adivinha se um filme é uma comédia ou um thriller usando apenas o número de vezes que palavras escolhidas aparecem no roteiro do filme. Ao final do projeto, você deverá saber como:

1. Construir um classificador k-vizinhos mais próximos.
2. Testar um classificador em dados.

### Logística

**Regras.** Não compartilhe seu código com ninguém além do seu parceiro, se você tiver um. Você pode discutir as questões com outros alunos, mas não compartilhe as respostas. A experiência de resolver os problemas neste projeto irá prepará-lo para os exames (e para a vida). Se alguém lhe pedir a resposta, resista! Em vez disso, você pode demonstrar como resolveria um problema semelhante.

**Suporte.** Você não está sozinho! Venha para as horas de atendimento e converse com seus colegas. Se você estiver se sentindo sobrecarregado ou não souber como progredir, envie um e-mail para obter ajuda.

**Testes.** Os testes fornecidos **não são abrangentes** e passar nos testes de uma questão **não significa** que você respondeu à questão corretamente. Os testes geralmente verificam apenas se sua tabela tem os rótulos de coluna corretos. No entanto, mais testes serão aplicados para verificar a correção da sua submissão para atribuir sua pontuação final, então seja cuidadoso e verifique seu trabalho! Você pode querer criar suas próprias verificações ao longo do caminho para ver se suas respostas fazem sentido. Além disso, antes de enviar, certifique-se de que nenhuma de suas células demore muito tempo para ser executada (vários minutos).

**Perguntas de Resposta Livre.** Certifique-se de colocar as respostas às perguntas escritas na célula indicada que fornecemos. **Toda pergunta de resposta livre deve incluir uma explicação** que responda adequadamente à pergunta. Seu trabalho escrito será carregado automaticamente no Gradescope após o prazo do projeto; não é necessária nenhuma ação da sua parte para isso.

**Conselho.** Desenvolva suas respostas de forma incremental. Para realizar uma tarefa complicada, divida-a em etapas, execute cada etapa em uma linha diferente, dê um novo nome a cada resultado e verifique se cada resultado intermediário é o que você espera. Você pode adicionar quaisquer nomes ou funções adicionais que desejar nas células fornecidas. Certifique-se de que está usando nomes de variáveis distintos e significativos ao longo do notebook. Nesse sentido, **NÃO** reutilize os nomes de variáveis que usamos ao avaliar suas respostas.

Você **nunca** precisa usar apenas uma linha neste projeto ou em qualquer outro. Use variáveis intermediárias e várias linhas tanto quanto desejar!

Todos os conceitos necessários para este projeto estão no livro didático. Se você estiver preso em um problema específico, ler a seção relevante do livro didático muitas vezes ajudará a esclarecer o conceito.

---

Para começar, carregue `datascience`, `numpy`, `plots` e `d8error`. Certifique-se também de executar a primeira célula deste notebook para carregar `otter`.

In [ ]:
# Execute esta célula para configurar o notebook, mas por favor, não a altere.
import numpy as np
import math
import datascience
from datascience import *

# Estas linhas configuram a funcionalidade e formatação de plotagem.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter("ignore")

import d8error

# Parte 1: O Conjunto de Dados

Neste projeto, estamos explorando roteiros de filmes. Tentaremos prever o gênero de cada filme a partir do texto do seu roteiro. Em particular, compilamos uma lista de 5.000 palavras que ocorrem em conversas entre personagens de filmes. Para cada filme, nosso conjunto de dados nos informa a frequência com que cada uma dessas palavras ocorre em certas conversas no seu roteiro. Todas as palavras foram convertidas para minúsculas.

Execute a célula abaixo para ler a tabela `movies`. **Pode levar até um minuto para carregar.**

In [ ]:
movies = Table.read_table('movies.csv')

Aqui está uma linha da tabela e algumas das frequências das palavras que foram ditas no filme.

In [ ]:
movies.where("Title", "runaway bride").select(0, 1, 2, 3, 4, 14, 49, 1042, 4004)

A célula acima imprime algumas colunas da linha do filme de comédia *Runaway Bride*. O filme contém 4895 palavras. A palavra "it" aparece 115 vezes, o que representa $\frac{115}{4895} \approx 0.0234092$ das palavras no filme. A palavra "england" não aparece.

Contexto adicional: Esta representação numérica de um corpo de texto, que descreve apenas as frequências de palavras individuais, é chamada de representação de saco de palavras (bag-of-words). Este é um modelo frequentemente usado em [NLP](https://en.wikipedia.org/wiki/Natural_language_processing). Muita informação é descartada nesta representação: a ordem das palavras, o contexto de cada palavra, quem disse o quê, o elenco de personagens e atores, etc. No entanto, uma representação de saco de palavras é frequentemente usada para aplicações de aprendizado de máquina como um ponto de partida razoável, porque uma grande quantidade de informação também é retida e expressa em um formato conveniente e compacto.

Neste projeto, investigaremos se essa representação é suficiente para construir um classificador de gênero preciso.

Todos os títulos de filmes são únicos. A função `row_for_title` fornece acesso rápido à única linha para cada título.

*Nota: Todos os filmes em nosso conjunto de dados têm seus títulos em letras minúsculas.*

In [ ]:
title_index = movies.index_by('Title')
def row_for_title(title):
    """Retorna a linha para um título, semelhante à expressão a seguir (mas mais rápido)
    
    movies.where('Title', title).row(0)
    """
    return title_index.get(title)[0]

row_for_title('toy story')

Por exemplo, a maneira mais rápida de encontrar a frequência de "fun" no filme *Toy Story* é acessar o item `'fun'` da sua linha. Verifique a tabela original para ver se isso funcionou para você!

In [ ]:
row_for_title('toy story').item('fun') 

#### Questão 1.0
Defina `expected_row_sum` para o número que você __espera__ que resulte da soma de todas as proporções em cada linha, excluindo as primeiras cinco colunas. Pense no que qualquer uma das linhas soma.

In [ ]:
# Defina row_sum como um número que é a soma (aproximada) de cada linha de proporções de palavras.
expected_row_sum = ...

In [ ]:
grader.check("q1_0")

Este conjunto de dados foi extraído de [um conjunto de dados da Universidade Cornell](http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html). Após transformar o conjunto de dados (por exemplo, convertendo as palavras para minúsculas, removendo palavras impróprias e convertendo as contagens em frequências), criamos este novo conjunto de dados contendo a frequência de 5000 palavras comuns em cada filme.

In [ ]:
print('Words with frequencies:', movies.drop(np.arange(5)).num_columns) 
print('Movies with genres:', movies.num_rows)

## 1.1. Radicalização de Palavras
As colunas, exceto "Title" (Título), "Year" (Ano), "Rating" (Classificação), "Genre" (Gênero) e "# Words" (Número de Palavras) na tabela `movies` são todas palavras que aparecem em alguns dos filmes do nosso conjunto de dados. Essas palavras foram *radicalizadas*, ou abreviadas heuristicamente, na tentativa de transformar diferentes formas [flexionadas](https://en.wikipedia.org/wiki/Inflection) da mesma palavra base na mesma string. Por exemplo, a coluna "manag" é a soma das proporções das palavras "manage" (gerenciar), "manager" (gerente), "managed" (gerenciado) e "managerial" (gerencial) (e talvez outras) em cada filme. Esta é uma técnica comum usada em aprendizado de máquina e processamento de linguagem natural.

A radicalização torna um pouco complicado procurar as palavras que você deseja usar, então fornecemos outra tabela chamada `vocab_table` que permitirá que você veja exemplos de versões não radicalizadas de cada palavra radicalizada. Execute o código abaixo para carregá-la.

**Nota:** Você deve usar `vocab_table` para o restante da Seção 1.1, não `vocab_mapping`.

In [ ]:
# Apenas execute esta célula
vocab_mapping = Table.read_table('stem.csv')
stemmed = np.take(movies.labels, np.arange(3, len(movies.labels)))
vocab_table = Table().with_column('Stem', stemmed).join('Stem', vocab_mapping)
vocab_table.take(np.arange(1100, 1110))

#### Questão 1.1.1
Usando `vocab_table`, encontre a versão radicalizada da palavra "elements" e atribua o valor a `stemmed_message`.

In [ ]:
stemmed_message = ...
stemmed_message

In [ ]:
grader.check("q1_1_1")

#### Questão 1.1.2
Qual radical no conjunto de dados tem mais palavras que são abreviadas para ele? Atribua `most_stem` a esse radical.

In [ ]:
most_stem = ...
most_stem

In [ ]:
grader.check("q1_1_2")

#### Questão 1.1.3
Qual é a palavra mais longa no conjunto de dados cujo radical não foi encurtado? Atribua isso a `longest_uncut`. Em caso de empate, ordene alfabeticamente de Z a A (então, se suas opções forem "gato" ou "rato", você deve escolher "rato"). Note que ao ordenar letras, a letra `a` é menor que a letra `z`.

*Dica 1:* `vocab_table` tem 2 colunas: uma para radicais e outra para a palavra não radicalizada (normal). Encontre a palavra mais longa que não foi cortada (mesmo comprimento que o radical).

*Dica 2:* Há uma função de tabela que permite calcular uma função em cada elemento de uma coluna. Verifique a [Referência Python](http://data8.org/sp22/python-reference.html) se não tiver certeza de qual usar.

*Dica 3:* Verifique os comentários na célula abaixo se estiver com dificuldades.

In [ ]:
# Em nossa solução, achamos útil primeiro adicionar colunas com
# o comprimento da palavra e o comprimento da raiz,
# e depois adicionar uma coluna com a diferença entre esses comprimentos.
# Qual será a diferença se a palavra não for encurtada?

tbl_with_lens = ...
tbl_with_diff = ...

longest_uncut = ...
longest_uncut

In [ ]:
grader.check("q1_1_3")

#### Questão 1.1.4
Quantos radicais têm apenas uma palavra que é abreviada para eles? Por exemplo, se o radical "book" (livro) mapeia apenas para a palavra "books" (livros) e se o radical "a" mapeia apenas para a palavra "a", ambos devem ser contados como radicais que mapeiam apenas para uma única palavra.

Atribua `count_single_stems` à contagem de radicais que mapeiam para apenas uma palavra.

In [ ]:
count_single_stems = ...
count_single_stems

In [ ]:
grader.check("q1_1_4")

## 1.2. Análise Exploratória de Dados: Regressão Linear

Vamos explorar nosso conjunto de dados antes de tentar construir um classificador. Para começar, usaremos as proporções associadas para investigar a relação entre diferentes palavras.

A primeira associação que investigaremos é a associação entre a proporção de palavras que são "outer" (externas) e a proporção de palavras que são "space" (espaço).

Como de costume, investigaremos nossos dados visualmente antes de realizar qualquer análise numérica.

Execute a célula abaixo para plotar um diagrama de dispersão das proporções de "space" (espaço) vs proporções de "outer" (externas) e para criar a tabela `outer_space`. Cada ponto no gráfico de dispersão representa um filme.

In [ ]:
# Apenas execute esta célula
outer_space = movies.select("outer", "space")
outer_space.scatter("outer", "space")
plots.axis([-0.0005, 0.001, -0.0005, 0.003]);
plots.xticks(rotation=45);

#### Questão 1.2.1
Olhando para esse gráfico, é difícil ver se há uma associação. Calcule o coeficiente de correlação para a possível associação linear entre a proporção de palavras que são "outer" (externas) e a proporção de palavras que são "space" (espaço) para cada filme no conjunto de dados, e atribua-o a `outer_space_r`.

*Dica:* Se você precisar de uma revisão sobre como calcular o coeficiente de correlação, confira [Capítulo 15.1](https://inferentialthinking.com/chapters/15/1/Correlation.html#calculating-r).

In [ ]:
# Esses dois arrays devem deixar seu código mais limpo!
outer = movies.column("outer")
space = movies.column("space")

outer_su = ...
space_su = ...

outer_space_r = ...
outer_space_r

In [ ]:
grader.check("q1_2_1")

<!-- BEGIN QUESTION -->

#### Questão 1.2.2
Escolha duas palavras *diferentes* no conjunto de dados com uma magnitude (valor absoluto) de correlação maior que 0.2 e plote um gráfico de dispersão com uma linha de melhor ajuste para elas. Por favor, não escolha "outer" e "space" ou "san" e "francisco". O código para plotar o gráfico de dispersão e a linha de melhor ajuste é fornecido para você, você só precisa calcular os valores corretos para `r`, `slope` e `intercept`.

*Dica 1:* É mais fácil pensar em palavras com correlação positiva, ou seja, palavras que são frequentemente mencionadas juntas. Tente pensar em frases comuns ou expressões idiomáticas.

*Dica 2:* Consulte a [Seção 15.2](https://inferentialthinking.com/chapters/15/2/Regression_Line.html#units-of-measurement-of-the-slope) do livro didático para as fórmulas.

In [ ]:
word_x = ...
word_y = ...

# Esses arrays devem deixar seu código mais limpo!
arr_x = movies.column(word_x)
arr_y = movies.column(word_y)

x_su = ...
y_su = ...

r = ...

slope = ...
intercept = ...

# NÃO ALTERE ESTA LINHA DE CÓDIGO
movies.scatter(word_x, word_y)
max_x = max(movies.column(word_x))
plots.title(f"Correlation: {r}, magnitude greater than .2: {abs(r) >= 0.2}")
plots.plot([0, max_x * 1.3], [intercept, intercept + slope * (max_x*1.3)], color='gold');

<!-- END QUESTION -->

#### Questão 1.2.3
Imagine que você escolheu as palavras "san" e "francisco" como as duas palavras que você esperaria que estivessem correlacionadas porque elas compõem o nome da cidade San Francisco. Atribua `san_francisco` ao número 1 ou ao número 2 de acordo com qual afirmação é verdadeira em relação à correlação entre "san" e "francisco".

1. "san" também pode preceder outros nomes de cidades como San Diego e San Jose. Isso pode levar a "san" aparecer em filmes sem "francisco" e reduzir a correlação entre "san" e "francisco."
2. "san" também pode preceder outros nomes de cidades como San Diego e San Jose. O fato de que "san" poderia aparecer mais frequentemente na frente de diferentes cidades e sem "francisco" aumentaria a correlação entre "san" e "francisco."




In [ ]:
san_francisco = ...

In [ ]:
grader.check("q1_2_3")

## 1.3. Dividindo o conjunto de dados
Agora, vamos usar nosso conjunto de dados `movies` para dois propósitos.

1. Primeiro, queremos *treinar* classificadores de gênero de filmes.
2. Segundo, queremos *testar* o desempenho de nossos classificadores.

Portanto, precisamos de dois conjuntos de dados diferentes: *treinamento* e *teste*.

O objetivo de um classificador é classificar dados não vistos que são semelhantes aos dados de treinamento. O conjunto de dados de teste nos ajudará a determinar a precisão de nossas previsões comparando os gêneros reais dos filmes com os gêneros que nosso classificador prevê. Portanto, devemos garantir que não haja filmes que apareçam em ambos os conjuntos. Fazemos isso dividindo o conjunto de dados aleatoriamente. O conjunto de dados já foi permutado aleatoriamente, então é fácil dividir. Basta pegar os primeiros 85% do conjunto de dados para treinamento e o restante para teste.

Execute o código abaixo (sem alterá-lo) para separar os conjuntos de dados em duas tabelas.

In [ ]:
# Aqui temos a proporção de nossos dados
# que queremos designar para treinamento como 17/20
# do nosso conjunto de dados total. 3/20 dos dados são
# reservados para teste.

training_proportion = 17/20

num_movies = movies.num_rows
num_train = int(num_movies * training_proportion)
num_test = num_movies - num_train

train_movies = movies.take(np.arange(num_train))
test_movies = movies.take(np.arange(num_train, num_movies))

print("Training: ",   train_movies.num_rows, ";",
      "Test: ",       test_movies.num_rows)

<!-- BEGIN QUESTION -->

#### Questão 1.3.1
Desenhe um gráfico de barras horizontal com duas barras que mostram a proporção de filmes de comédia em cada conjunto de dados (`train_movies` e `test_movies`). As duas barras devem ser rotuladas como "Treinamento" e "Teste". Complete a função `comedy_proportion` primeiro; ela deve ajudá-lo a criar o gráfico de barras.

*Dica*: Consulte a [Seção 7.1](https://inferentialthinking.com/chapters/07/1/Visualizing_Categorical_Distributions.html#bar-chart) do livro didático se precisar de uma revisão sobre gráficos de barras.




In [ ]:
def comedy_proportion(table):
    # Retorna a proporção de filmes em uma tabela que têm o gênero comédia.
    ...
    return ...

# A solução da equipe levou várias linhas. Comece criando uma tabela.
# Se você ficar preso, pense sobre que tipo de tabela você precisa para o barh funcionar
...

<!-- END QUESTION -->

# Parte 2: K-Nearest Neighbors - Um Exemplo Guiado

[K-Nearest Neighbors (k-NN)](https://inferentialthinking.com/chapters/17/1/Nearest_Neighbors.html) é um algoritmo de classificação. Dadas algumas *atribuições* numéricas (também chamadas de *características*) de um exemplo não visto, ele decide a qual categoria esse exemplo pertence com base em sua similaridade com exemplos previamente vistos. Prever a categoria de um exemplo é chamado de *rotulagem*, e a categoria prevista também é chamada de *rótulo*.

Uma característica (atributo) que temos sobre cada filme é *a proporção de vezes que uma determinada palavra aparece no filme*, e os rótulos são dois gêneros de filmes: comédia e thriller. O algoritmo requer muitos exemplos previamente vistos para os quais tanto as características quanto os rótulos são conhecidos: essa é a tabela `train_movies`.

Para construir o entendimento, vamos visualizar o algoritmo em vez de apenas descrevê-lo.

## 2.1. Classificando um filme

No k-NN, classificamos um filme encontrando os `k` filmes no *conjunto de treinamento* que são mais semelhantes de acordo com as características que escolhemos. Chamamos esses filmes com características semelhantes de *vizinhos mais próximos*. O algoritmo k-NN atribui o filme à categoria mais comum entre seus `k` vizinhos mais próximos.

Vamos nos limitar a apenas 2 características por enquanto, para que possamos plotar cada filme. As características que usaremos são as proporções das palavras "water" (água) e "feel" (sentir) no filme. Pegando o filme *Monty Python and the Holy Grail* (no conjunto de teste), 0.000804074 de suas palavras são "water" e 0.0010721 são "feel". Este filme aparece no conjunto de teste, então vamos imaginar que ainda não sabemos seu gênero.

Primeiro, precisamos tornar nossa noção de similaridade mais precisa. Diremos que a *distância* entre dois filmes é a distância em linha reta entre eles quando plotamos suas características em um diagrama de dispersão.

**Essa distância é chamada de distância Euclidiana, cuja fórmula é $\sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}$.**

Por exemplo, no filme *Clerks.* (no conjunto de treinamento), 0.00016293 de todas as palavras no filme são "water" e 0.00154786 são "feel". Sua distância de *Monty Python and the Holy Grail* neste conjunto de características de 2 palavras é $\sqrt{(0.000804074 - 0.000162933)^2 + (0.0010721 - 0.00154786)^2} \approx 0.000798379$. (Se incluíssemos mais ou diferentes características, a distância poderia ser diferente.)

Um terceiro filme, *The Godfather* (no conjunto de treinamento), tem 0 "water" e 0.00015122 "feel".

A função abaixo cria um gráfico para exibir as características "water" e "feel" de um filme de teste e alguns filmes de treinamento. Como você pode ver no resultado, *Monty Python and the Holy Grail* é mais semelhante a *Clerks.* do que a *The Godfather* com base nessas características, o que faz sentido, pois ambos os filmes são comédias, enquanto *The Godfather* é um thriller.

In [ ]:
# Apenas execute esta célula
def plot_with_two_features(test_movie, training_movies, x_feature, y_feature):
    """Plot um filme de teste e filmes de treinamento usando duas features."""
    test_row = row_for_title(test_movie)
    distances = Table().with_columns(
            x_feature, [test_row.item(x_feature)],
            y_feature, [test_row.item(y_feature)],
            'Color',   ['unknown'],
            'Title',   [test_movie]
        )
    for movie in training_movies:
        row = row_for_title(movie)
        distances.append([row.item(x_feature), row.item(y_feature), row.item('Genre'), movie])
    distances.scatter(x_feature, y_feature, group='Color', labels='Title', s=50)
    
training = ["clerks.", "the godfather"] 
plot_with_two_features("monty python and the holy grail", training, "water", "feel")
plots.axis([-0.0008, 0.001, -0.004, 0.007]);

#### Questão 2.1.1

Calcule a distância Euclidiana (definida na seção acima) entre os dois filmes, *Monty Python and the Holy Grail* e *The Godfather*, usando apenas as características `water` e `feel`. Atribua o nome `one_distance` a essa distância.

*Dica 1:* Se você tiver uma linha, pode usar `item` para obter um valor de uma coluna pelo seu nome. Por exemplo, se `r` é uma linha, então `r.item("Genre")` é o valor na coluna `"Genre"` na linha `r`.

*Dica 2:* Consulte o início da Parte 1 se não se lembrar do que `row_for_title` faz.

*Dica 3:* Na fórmula para a distância Euclidiana, pense cuidadosamente sobre o que `x` e `y` representam. Consulte o exemplo no texto acima se estiver em dúvida.

In [ ]:
python = row_for_title("monty python and the holy grail") 
godfather = row_for_title("the godfather") 

one_distance = ...
one_distance

In [ ]:
grader.check("q2_1_1")

Abaixo, adicionamos um terceiro filme de treinamento, *The Silence of the Lambs* (O Silêncio dos Inocentes). Antes, o ponto mais próximo de *Monty Python and the Holy Grail* era *Clerks.*, um filme de comédia. No entanto, agora o ponto mais próximo é *The Silence of the Lambs*, um filme de suspense.

In [ ]:
training = ["clerks.", "the godfather", "the silence of the lambs"] 
plot_with_two_features("monty python and the holy grail", training, "water", "feel") 
plots.axis([-0.0008, 0.001, -0.004, 0.007]);

#### Questão 2.1.2
Complete a função `distance_two_features` que calcula a distância Euclidiana entre quaisquer dois filmes, usando duas características. As duas últimas linhas chamam sua função para mostrar que *Monty Python and the Holy Grail* está mais próximo de *The Silence of the Lambs* do que de *Clerks*.

In [ ]:
def distance_two_features(title0, title1, x_feature, y_feature):
    """Calcula a distância entre dois filmes com títulos title0 e title1.
    
    Apenas as características nomeadas x_feature e y_feature são usadas ao calcular a distância.
    """

    row0 = ...
    row1 = ...
    ...

for movie in make_array("clerks.", "the silence of the lambs"):
    movie_distance = distance_two_features(movie, "monty python and the holy grail", "water", "feel")
    print(movie, 'distance:\t', movie_distance)

In [ ]:
grader.check("q2_1_2")

#### Questão 2.1.3
Defina a função `distance_from_python` para que funcione conforme descrito em sua documentação.

**Nota:** Sua solução não deve usar operações aritméticas diretamente. Em vez disso, deve fazer uso da funcionalidade existente acima!

In [ ]:
def distance_from_python(title):
    """A distância entre o filme dado e "monty python and the holy grail", 
    com base nas características "water" e "feel".
    
    Esta função recebe um único argumento:
      title: Uma string, o nome de um filme.
    """
    
    ...

# Calcule a distância entre "Clerks." e "Monty Python and the Holy Grail"
distance_from_python('clerks.')

In [ ]:
grader.check("q2_1_3")

#### Questão 2.1.4

Usando as características `"water"` e `"feel"`, quais são os nomes e gêneros dos 5 filmes no **conjunto de treinamento** mais próximos de *Monty Python and the Holy Grail*? Para responder a esta pergunta, faça uma **tabela** chamada `close_movies` contendo esses 5 filmes com as colunas `"Title"`, `"Genre"`, `"water"`, e `"feel"`, assim como uma coluna chamada `"distance from python"` que contém a distância de *Monty Python and the Holy Grail*. A tabela deve estar **ordenada em ordem crescente por `"distance from python"`**.

*Nota:* Por que distâncias menores de *Monty Python and the Holy Grail* são mais úteis para nos ajudar a classificar o filme?

*Dica:* Sua tabela final deve ter apenas 5 linhas. Como você pode obter as primeiras cinco linhas de uma tabela?

In [ ]:

# A solução da equipe levou várias linhas.
...
close_movies = ...
close_movies

In [ ]:
grader.check("q2_1_4")

#### Questão 2.1.5
Em seguida, classificaremos *Monty Python and the Holy Grail* com base nos gêneros dos filmes mais próximos.

Para isso, defina a função `most_common` para que funcione conforme descrito em sua documentação abaixo.

In [ ]:
def most_common(label, table):
    """O elemento mais comum em uma coluna de uma tabela.
    
    Esta função recebe dois argumentos:
      label: O rótulo de uma coluna, uma string.
      table: Uma tabela.
     
    Ela retorna o valor mais comum na coluna label da tabela.
    Em caso de empate, ela retorna qualquer um dos valores mais comuns.    
    """
    ...

# Chamar most_common na sua tabela dos 5 vizinhos mais próximos classifica
# "monty python and the holy grail" como um filme de suspense, 3 votos a 2.
most_common('Genre', close_movies)

In [ ]:
grader.check("q2_1_5")

Parabéns são merecidos -- você classificou seu primeiro filme! No entanto, podemos ver que o classificador não funciona muito bem, pois categorizou *Monty Python and the Holy Grail* como um filme de suspense (a menos que você considere a cena emocionante da granada sagrada). Vamos ver se podemos fazer melhor!

# Parabéns

<img src="opo1.jpeg" alt="drawing" width="300"/>

Opo quer parabenizá-lo por alcançar o ponto de verificação!

Seu instrutor pode querer que você envie seu trabalho na Parte 1 como um ponto de verificação do seu progresso.

Execute as células que Opo está olhando.

<img src="opo.jpeg" alt="drawing" width="300"/>

In [ ]:
checkpoint_tests = ["q1_0", "q1_1_1","q1_1_2","q1_1_3","q1_1_4","q1_2_1","q1_2_3",
                    "q2_1_1","q2_1_2","q2_1_3","q2_1_4","q2_1_5"]
for test in checkpoint_tests:
    display(grader.check(test))

## Submissão
Se o seu instrutor gostaria que você enviasse o trabalho na parte um como um ponto de verificação do projeto, siga as instruções abaixo.

Certifique-se de ter executado todas as células no seu notebook em ordem antes de executar a célula abaixo, para que todas as imagens/gráficos apareçam na saída. Você pode fazer isso indo em `Cell > Run All`. A célula abaixo gerará um arquivo zip para você enviar. **Por favor, salve antes de exportar!**

**Lembretes:**
- Certifique-se de esperar até que o autograder termine de rodar para garantir que sua submissão foi processada corretamente e que você enviou para a tarefa correta.

In [ ]:
# Salve seu notebook primeiro, depois execute esta célula para exportar sua submissão.
grader.export(pdf=False, force_save=True)

# Parte 3: Características

Agora, vamos estender nosso classificador para considerar mais de duas características ao mesmo tempo.

A distância Euclidiana ainda faz sentido com mais de duas características. Para `n` características diferentes, calculamos a diferença entre os valores correspondentes das características de dois filmes, elevamos ao quadrado cada uma das `n` diferenças, somamos os números resultantes e tiramos a raiz quadrada da soma.

#### Questão 3.0
Escreva uma função chamada `distance` para calcular a distância Euclidiana entre dois **arrays** de características **numéricas** (por exemplo, arrays das proporções de vezes que diferentes palavras aparecem). A função deve ser capaz de calcular a distância Euclidiana entre dois arrays de comprimento arbitrário (mas igual).

Em seguida, use a função que você acabou de definir para calcular a distância **entre o primeiro e o segundo filme** no **conjunto de treinamento** *usando todas as características*. (Lembre-se de que as primeiras cinco colunas das suas tabelas não são características.)

*Dica 1:* Para converter linhas em arrays, use `np.array`. Por exemplo, se `t` fosse uma tabela, `np.array(t.row(0))` converte a linha 0 de `t` em um array.

*Dica 2:* Certifique-se de remover as primeiras cinco colunas da tabela antes de calcular `distance_first_to_second`, pois essas colunas não contêm nenhuma característica (as proporções das palavras).

In [ ]:
def distance(features_array1, features_array2):
    """A distância Euclidiana entre dois arrays de valores de características."""
    ...

distance_first_to_second = ...
distance_first_to_second

In [ ]:
grader.check("q3_0")

## 3.1. Criando seu próprio conjunto de características

Infelizmente, usar todas as características tem algumas desvantagens. Uma desvantagem clara é a falta de *eficiência computacional* -- calcular distâncias Euclidianas leva muito tempo quando temos muitas características. Você pode ter notado isso na última questão!

Então, vamos selecionar apenas 20. Gostaríamos de escolher características que sejam muito *discriminativas*. Ou seja, características que nos levem a classificar corretamente a maior parte do conjunto de teste possível. Esse processo de escolher características que farão um classificador funcionar bem às vezes é chamado de *seleção de características*, ou, mais amplamente, *engenharia de características*.

Nesta questão, vamos ajudá-lo a começar a selecionar características mais eficazes para distinguir filmes de comédia de filmes de suspense. O gráfico abaixo (gerado para você) mostra o número médio de vezes que cada palavra ocorre em um filme de comédia no eixo horizontal e o número médio de vezes que ocorre em um filme de suspense no eixo vertical.

*Nota: A linha traçada é a linha de melhor ajuste, NÃO a linha y=x.*

![alt text](word_plot.png "Title")

As questões 3.1.1 a 3.1.4 pedirão que você interprete o gráfico acima. Para cada questão, selecione uma das seguintes opções e atribua seu número ao nome fornecido.
1. A palavra é comum tanto em filmes de comédia quanto em filmes de suspense.
2. A palavra é incomum em filmes de comédia e comum em filmes de suspense.
3. A palavra é comum em filmes de comédia e incomum em filmes de suspense.
4. A palavra é incomum tanto em filmes de comédia quanto em filmes de suspense.
5. Não é possível dizer a partir do gráfico.

**Questão 3.1.1**

Quais propriedades as palavras no canto inferior esquerdo do gráfico possuem? Sua resposta deve ser um único número de 1 a 5, correspondente à afirmação correta das opções acima.

In [ ]:
bottom_left = ...

In [ ]:
grader.check("q3_1_1")

**Questão 3.1.2**

Quais propriedades as palavras no canto inferior direito possuem?

In [ ]:
bottom_right = ...

In [ ]:
grader.check("q3_1_2")

**Questão 3.1.3**

Quais propriedades as palavras no canto superior direito possuem?

In [ ]:
top_right = ...

In [ ]:
grader.check("q3_1_3")

**Questão 3.1.4**

Quais propriedades as palavras no canto superior esquerdo possuem?

In [ ]:
top_left = ...

In [ ]:
grader.check("q3_1_4")

**Questão 3.1.5**

Se virmos um filme com muitas palavras que são comuns em filmes de comédia, mas incomuns em filmes de suspense, qual seria um palpite razoável sobre o gênero do filme? Atribua `movie_genre` ao número inteiro correspondente à sua resposta:
1. É um filme de suspense.
2. É um filme de comédia.

In [ ]:
movie_genre_guess = ...

In [ ]:
grader.check("q3_1_5")

#### Questão 3.1.6
Usando o gráfico acima, faça um array de pelo menos 10 palavras comuns que você acha que podem permitir que você **distinga** entre filmes de comédia e suspense. Certifique-se de escolher palavras que sejam **frequentes o suficiente** para que cada filme contenha pelo menos uma delas. Não escolha apenas as palavras mais frequentes -- você pode fazer muito melhor.

In [ ]:
# Defina my_features como um array de pelo menos 10 características (strings que são rótulos de colunas)

my_features = ...

# Selecione as 10 características de interesse tanto do conjunto de treino quanto do conjunto de teste
train_my_features = train_movies.select(my_features)
test_my_features = test_movies.select(my_features)

In [ ]:
grader.check("q3_1_6")

Este teste garante que você escolheu palavras de modo que pelo menos uma apareça em cada filme. Se você não conseguir encontrar palavras que satisfaçam este teste apenas pela intuição, tente escrever um código para imprimir os títulos dos filmes que não contêm nenhuma palavra da sua lista e, em seguida, veja as palavras que eles contêm.

<!-- BEGIN QUESTION -->

#### Questão 3.1.7
Em duas frases ou menos, descreva como você selecionou suas características.

_Digite sua resposta aqui, substituindo este texto._

<!-- END QUESTION -->

Em seguida, vamos classificar o primeiro filme do nosso conjunto de teste usando essas características. Você pode examinar o filme executando as células abaixo. Você acha que ele será classificado corretamente?

In [ ]:
print("Movie:")
test_movies.take(0).select('Title', 'Genre').show()
print("Features:")
test_my_features.take(0).show()

Como antes, queremos procurar os filmes no conjunto de treinamento que são mais parecidos com nosso filme de teste. Calcularemos as distâncias Euclidianas do filme de teste (usando `my_features`) para todos os filmes no conjunto de treinamento. Você poderia fazer isso com um `for` loop, mas para tornar isso computacionalmente mais rápido, fornecemos uma função, `fast_distances`, para fazer isso por você. Leia sua documentação para garantir que você entenda o que ela faz. (Você não precisa entender o código em seu corpo, a menos que queira.)

In [ ]:
# Just run this cell to define fast_distances.

def fast_distances(test_row, train_table):
    """Retorna um array das distâncias entre test_row e cada linha em train_table.

    Recebe 2 argumentos:
      test_row: Uma linha de uma tabela contendo características de um
        filme de teste (por exemplo, test_my_features.row(0)).
      train_table: Uma tabela de características (por exemplo, a tabela inteira
        train_my_features)."""

    assert train_table.num_columns < 50, "Certifique-se de que você não está usando todas as características da tabela de filmes."
    assert type(test_row) != datascience.tables.Table, "Certifique-se de que você está passando um objeto de linha para fast_distances."
    assert len(test_row) == len(train_table.row(0)), "Certifique-se de que o comprimento da linha de teste é o mesmo que o comprimento de uma linha em train_table."

    counts_matrix = np.asmatrix(train_table.columns).transpose()
    diff = np.tile(np.array(list(test_row)), [counts_matrix.shape[0], 1]) - counts_matrix
    np.random.seed(0) # Para fins de desempate
    distances = np.squeeze(np.asarray(np.sqrt(np.square(diff).sum(1))))
    eps = np.random.uniform(size=distances.shape)*1e-10 # Ruído para desempate
    distances = distances + eps
    return distances

#### Questão 3.1.8
Use a função `fast_distances` fornecida acima para calcular a distância do primeiro filme no seu conjunto de teste para todos os filmes no conjunto de treinamento, **usando seu conjunto de características**. Crie uma nova tabela chamada `genre_and_distances` com uma linha para cada filme no conjunto de treinamento e duas colunas:
* O `"Genre"` do filme de treinamento
* A `"Distance"` do primeiro filme no conjunto de teste

Certifique-se de que `genre_and_distances` esteja **ordenada em ordem crescente pela distância até o primeiro filme de teste**.

*Dica:* Pense em como você pode usar as variáveis que definiu na questão 3.1.6.

In [ ]:
# A solução da equipe levou várias linhas de código.
genre_and_distances = ...
genre_and_distances

In [ ]:
grader.check("q3_1_8")

#### Questão 3.1.9
Agora, calcule a classificação dos 7 vizinhos mais próximos do primeiro filme no conjunto de teste. Ou seja, decida seu gênero encontrando o gênero mais comum entre seus 7 vizinhos mais próximos no conjunto de treinamento, de acordo com as distâncias que você calculou. Em seguida, verifique se o seu classificador escolheu o gênero correto. (Dependendo das características que você escolheu, seu classificador pode não acertar este filme, e tudo bem.)

*Dica:* Você deve usar a função `most_common` que foi definida anteriormente.

In [ ]:
# Defina my_assigned_genre como o gênero mais comum entre estes.
my_assigned_genre = ...

# Defina my_assigned_genre_was_correct como True se my_assigned_genre
# corresponder ao gênero real do primeiro filme no conjunto de teste, False caso contrário.
my_assigned_genre_was_correct = ...

print("The assigned genre, {}, was{}correct.".format(my_assigned_genre, " " if my_assigned_genre_was_correct else " not "))

In [ ]:
grader.check("q3_1_9")

## 3.2. Uma função de classificador

Agora podemos escrever uma única função que encapsula todo o processo de classificação.

#### Questão 3.2.1
Escreva uma função chamada `classify`. Ela deve receber os seguintes quatro argumentos:
* Uma linha de características para um filme a ser classificado (por exemplo, `test_my_features.row(0)`).
* Uma tabela com uma coluna para cada característica (por exemplo, `train_my_features`).
* Um array de classes (por exemplo, os rótulos "comédia" ou "suspense") que tenha tantos itens quanto a tabela anterior tem linhas, e na mesma ordem. *Dica:* Quais são os rótulos de cada linha no conjunto de treinamento?
* `k`, o número de vizinhos a serem usados na classificação.

Ela deve retornar a classe que um classificador de `k`-vizinhos mais próximos escolhe para a linha de características fornecida (a string `'comedy'` ou a string `'thriller'`).

In [ ]:
def classify(test_row, train_features, train_labels, k):
    """Retorna a classe mais comum entre os k vizinhos mais próximos de test_row."""
    distances = fast_distances(test_row, train_features)
    genre_and_distances = ...
    ...

In [ ]:
grader.check("q3_2_1")

#### Questão 3.2.2

Atribua `godzilla_genre` ao gênero previsto pelo seu classificador para o filme "godzilla" no conjunto de teste, usando **15 vizinhos** e usando suas 10 características.

*Dica:* A função `row_for_title` não funcionará aqui.

In [ ]:
# A solução da equipe primeiro definiu uma linha chamada godzilla_features.
godzilla_features = ...
godzilla_genre = ...
godzilla_genre

In [ ]:
grader.check("q3_2_2")

Finalmente, quando avaliarmos nosso classificador, será útil ter uma função de classificação especializada para usar um conjunto de treinamento fixo e um valor fixo de `k`.

#### Questão 3.2.3
Crie uma função de classificação que receba como argumento uma linha contendo suas 10 características e classifique essa linha usando o algoritmo dos 15 vizinhos mais próximos com `train_my_features` como seu conjunto de treinamento.

In [ ]:
def classify_feature_row(row):
    ...

# Quando você terminar, isso deve produzir 'thriller' ou 'comedy'.
classify_feature_row(test_my_features.row(0))

In [ ]:
grader.check("q3_2_3")

## 3.3. Avaliando seu classificador

Agora que é fácil usar o classificador, vamos ver quão preciso ele é em todo o conjunto de teste.

**Questão 3.3.1**

Use `classify_feature_row` e `apply` para classificar todos os filmes no conjunto de teste. Atribua essas previsões como um array a `test_guesses`. Em seguida, calcule a proporção de classificações corretas.

*Dica 1*: Se você não especificar nenhuma coluna em `tbl.apply(...)`, sua função será aplicada a cada objeto de linha em `tbl`.

*Dica 2*: Em qual conjunto de dados você quer aplicar essa função?

In [ ]:
test_guesses = ...
proportion_correct = ...
proportion_correct

In [ ]:
grader.check("q3_3_1")

**Questão 3.3.2**

Uma parte importante da avaliação dos seus classificadores é descobrir onde eles cometem erros. Atribua o nome `test_movie_correctness` a uma tabela com três colunas: `'Title'`, `'Genre'` e `'Was correct'`.

- A coluna `'Genre'` deve conter os gêneros originais, não os que você previu.
- A coluna `'Was correct'` deve conter `True` ou `False`, dependendo se o filme foi classificado corretamente ou não.

In [ ]:
# Sinta-se à vontade para usar várias linhas de código
# mas certifique-se de atribuir test_movie_correctness à tabela correta!
test_movie_correctness = ...
test_movie_correctness.sort('Was correct', descending = True).show(5)

In [ ]:
grader.check("q3_3_2")

<!-- BEGIN QUESTION -->

**Questão 3.3.3**

Você vê um padrão nos tipos de filmes que seu classificador classifica incorretamente? Em duas frases ou menos, descreva quaisquer padrões que você veja nos resultados ou quaisquer outras descobertas interessantes da tabela acima. Se precisar de ajuda, tente procurar os filmes que seu classificador errou na Wikipedia.

_Digite sua resposta aqui, substituindo este texto._

<!-- END QUESTION -->

Neste ponto, você passou por um ciclo de design de classificador. Vamos resumir os passos:
1. A partir dos dados disponíveis, selecione conjuntos de teste e treinamento.
2. Escolha um algoritmo que você vai usar para a classificação k-NN.
3. Identifique algumas características.
4. Defina uma função de classificador usando suas características e o conjunto de treinamento.
5. Avalie seu desempenho (a proporção de classificações corretas) no conjunto de teste.

# Parte 4: Explorações
Agora que você sabe como avaliar um classificador, é hora de construir outro.

Seus amigos são grandes fãs de filmes de comédia e suspense. Eles criaram seu próprio conjunto de dados de filmes que querem assistir, mas precisam da sua ajuda para determinar o gênero de cada filme em seu conjunto de dados (comédia ou suspense). Você nunca viu nenhum dos filmes no conjunto de dados dos seus amigos, então nenhum dos filmes dos seus amigos está presente no seu conjunto de treinamento ou teste anterior. Em outras palavras, este novo conjunto de dados de filmes pode funcionar como outro conjunto de teste no qual faremos previsões com base em nossos dados de treinamento originais.

Execute a célula a seguir para carregar os dados dos filmes dos seus amigos.

In [ ]:
friend_movies = Table.read_table('friend_movies.csv')
friend_movies.show(5)

**Questão 4.1**

O computador do seu amigo não é tão poderoso quanto o seu, então ele diz que o classificador que você criar para ele pode ter no máximo 5 palavras como características. Desenvolva um novo classificador com a restrição de **usar no máximo 5 características.** Atribua `new_features` a um array com suas características.

Sua nova função deve ter os mesmos argumentos que `classify_feature_row` e retornar uma classificação. Nomeie-a `another_classifier`. Em seguida, exiba sua precisão usando o código anterior para comparar o novo classificador com o antigo.

Algumas maneiras de você mudar seu classificador são usando diferentes características ou tentando diferentes valores de `k`. (Claro, você ainda tem que usar `train_movies` como seu conjunto de treinamento!)

**Certifique-se de não reatribuir nenhuma variável usada anteriormente aqui**, como `proportion_correct` da questão anterior.

*Nota:* Não há uma única maneira correta de fazer isso! Apenas certifique-se de que você pode explicar seu raciocínio por trás das novas escolhas.

In [ ]:
new_features = ...

train_new = train_movies.select(new_features)
test_new = friend_movies.select(new_features)

def another_classifier(row):
    ...

In [ ]:
grader.check("q4_1")

<!-- BEGIN QUESTION -->

**Questão 4.2**

Você vê um padrão nos erros que seu novo classificador comete? Qual foi a precisão que você conseguiu obter com seu classificador limitado? Você notou uma melhoria do seu primeiro classificador para o segundo? Descreva em duas frases ou menos.

*Dica:* Pode ser que você não consiga ver um padrão.


_Digite sua resposta aqui, substituindo este texto._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Questão 4.3**

Dada a restrição de cinco palavras, como você selecionou essas cinco? Descreva em duas frases ou menos.


_Digite sua resposta aqui, substituindo este texto._

<!-- END QUESTION -->

# Parte 5: Outros Métodos de Classificação (OPCIONAL)

**Nota**: Tudo abaixo é **OPCIONAL**. Por favor, trabalhe nesta parte apenas depois de ter terminado e enviado o projeto. Se você criar novas células abaixo, NÃO reatribua variáveis definidas em partes anteriores do projeto.

Agora que você terminou seu classificador k-NN, pode estar se perguntando o que mais poderia fazer para melhorar sua precisão no conjunto de teste. A classificação é uma das muitas tarefas de aprendizado de máquina, e há muitos outros algoritmos de classificação! Se você se sentir inclinado, encorajamos você a tentar qualquer método que possa ajudar a melhorar seu classificador.

Compilamos uma lista de postagens de blog com mais informações sobre classificação e aprendizado de máquina. Crie quantas células quiser abaixo -- você pode usá-las para importar novos módulos ou implementar novos algoritmos.

Postagens de blog:

* [Algoritmos/métodos de classificação](https://medium.com/@sifium/machine-learning-types-of-classification-9497bd4f2e14)
* [Divisão de treino/teste e validação cruzada](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6)
* [Mais informações sobre k-vizinhos mais próximos](https://medium.com/@adi.bronshtein/a-quick-introduction-to-k-nearest-neighbors-algorithm-62214cea29c7)
* [Overfitting](https://elitedatascience.com/overfitting-in-machine-learning)

Em futuras aulas de ciência de dados, como Ciência de Dados 100, você aprenderá sobre alguns dos algoritmos nas postagens de blog acima, incluindo regressão logística. Você também aprenderá mais sobre overfitting, validação cruzada e abordagens para diferentes tipos de problemas de aprendizado de máquina.

Um módulo que você pode considerar usar é o [Scikit-learn](http://scikit-learn.org/stable/tutorial/basic/tutorial.html). Há muito a considerar, então encorajamos você a encontrar mais informações por conta própria!

In [ ]:
...

# Yoshi quer dizer: Parabéns! Você terminou o Projeto 3.

<img src="yoshi.jpeg" alt="desenho" width="300"/>

Usando suas habilidades em estatística e aprendizado de máquina, você:
- Investigou um conjunto de dados de roteiros de filmes
- Identificou associações de palavras
- Construiu um modelo de aprendizado de máquina para classificar filmes pelos seus roteiros

Você terminou! Hora de enviar.

**Passos importantes para a submissão:**
1. Execute os testes e verifique se todos passam.
2. Escolha **Salvar Notebook** no menu **Arquivo**, depois **execute a célula final**.
3. Clique no link para baixar o arquivo zip.
4. Em seguida, envie o arquivo zip para a tarefa correspondente de acordo com as instruções do seu instrutor.

**É sua responsabilidade garantir que seu trabalho esteja salvo antes de executar a última célula.**

## Submissão

Certifique-se de ter executado todas as células no seu notebook em ordem antes de executar a célula abaixo, para que todas as imagens/gráficos apareçam na saída. A célula abaixo gerará um arquivo zip para você enviar. **Por favor, salve antes de exportar!**

In [ ]:
# Salve seu notebook primeiro, depois execute esta célula para exportar sua submissão.
grader.export(pdf=False, run_tests=True)